In [215]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import urllib2
from sklearn.linear_model import LogisticRegression

%matplotlib inline
import matplotlib.pyplot as plt

import statsmodels.api as sm
import pylab as pl
import numpy as np
from sklearn import cross_validation
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors, datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [62]:
urlForHigh = "http://www.indeed.com/jobs?q=data+scientist+%24130%2C000&l="
urlForLow  = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l="
page_start_from = '&start='    


df1 = pd.DataFrame()    
df2 = pd.DataFrame()   

for city in ('New+York%2C+NY', 'Chicago%2C+IL', 'San+Francisco%2C+CA', 'Austin%2C+TX',
            "Atlanta%2C+GA", "Los+Angeles%2C+CA", "Washington%2C+DC","Denver","Seattle","Austin","Dallas","San+Jose","Boston","Charlotte" ):
    for page in range(1,21): 
        page = (page-1) * 10  
        urlL = "%s%s%s%d" % (urlForLow, city, page_start_from, page) 
        r = requests.get(urlL)
        soupL = BeautifulSoup(r.content) 

        results = soupL.find_all("div", {'class' : ' row result'}) 
    
        for i in range(len(results)):
            title    = results[i].find ("a", {"class": "turnstileLink"}).get_text().strip()
            company  = results[i].find ("span", {"class": "company"}).get_text().strip()
            location = results[i].find ("span", {"class": "location"}).get_text().strip()
            summary  = results[i].find ("span", {"class": "summary"}).get_text().strip()        
        
        
            df1 = df1.append({"Company Name": company, "Job Title": title, "Location": location,
                            "Job Summary": summary}, ignore_index=True)

    

for city in ('New+York%2C+NY', 'Chicago%2C+IL', 'San+Francisco%2C+CA', 'Austin%2C+TX',
            "Atlanta%2C+GA", "Los+Angeles%2C+CA", "Washington%2C+DC","Denver","Seattle","Austin","Dallas","San+Jose","Boston","Charlotte" ):
    for page in range(1,21): 
        page = (page-1) * 10  
        urlH = "%s%s%s%d" % (urlForHigh, city, page_start_from, page) 
        r = requests.get(urlH)
        soupH = BeautifulSoup(r.content) 

        results = soupH.find_all("div", {'class' : ' row result'}) 
    
        for i in range(len(results)):
            title    = results[i].find ("a", {"class": "turnstileLink"}).get_text().strip()
            company  = results[i].find ("span", {"class": "company"}).get_text().strip()
            location = results[i].find ("span", {"class": "location"}).get_text().strip()
            summary  = results[i].find ("span", {"class": "summary"}).get_text().strip()        
           
        
            df2 = df2.append({"Company Name": company, "Job Title": title, "Location": location,
                            "Job Summary": summary}, ignore_index=True)


KeyboardInterrupt: 

In [ ]:
print df1.shape
print df2.shape

In [ ]:
df1["Salary Class"]="Low" 
df1["Class"]=0
df2["Salary Class"]='High'
df2["Class"]=1

In [ ]:
merge=[df1,df2]

In [ ]:
merged = pd.concat(merge)

In [ ]:
df3=merged.drop_duplicates(["Company Name","Job Title","Location","Job Summary"],take_last=True)

In [ ]:
df3.to_csv("data.csv", encoding='utf-8')

In [ ]:
data=pd.read_csv('data.csv')

In [ ]:
del data['Unnamed: 0']

In [ ]:
data

In [ ]:
def binJob1(job):
    jobTitle = job.lower()
    if "data scientist" in jobTitle:
        return "Data scientist"
    elif "engineer" in jobTitle:
        return "Engineer"
    elif "marketing" in jobTitle:
        return "Marketing"
    elif "machine learning" in jobTitle:
        return "Machine learning"
    elif "software" in jobTitle:
        return "Software"
    elif "research" in jobTitle:
        return "Research"
    elif "consultant"in jobTitle:
        return "Consultant"
    elif "cyber" in jobTitle:
        return "Cyber"
    else:
        return "Other"
    
data["JobFunction"] = data["Job Title"].apply(binJob1)

In [ ]:
def binJob2(job):
    jobTitle = job.lower()
    
    
    if ("sr" in jobTitle) | ("senior" in jobTitle):
        return 'Senior'
    elif "manager" in jobTitle:
        return "Manager"
    elif "principle" in jobTitle:
        return "Principle"
    elif "chief" in jobTitle:
        return "Chief"
    elif "lead" in jobTitle:
        return "Lead"
    elif "director"in jobTitle:
        return "Director"
    elif "entry" in jobTitle:
        return "Entry"
    elif "fellow" in jobTitle:
        return "Fellow"
    elif "assistant" in jobTitle:
        return "Assistant"
    else:
        return "Other"
    
data["JobSeniority"] = data["Job Title"].apply(binJob2)

In [ ]:

def locationToRegion(location):
    if ("NY" in location)|("MA" in location)|("NJ" in location):
        return "Northeast"
    elif ("DC" in location)|("VA" in location)|("MD" in location):
        return "DMV"
    elif "CA"  in location:
        return "California"
    elif ("IL" in location)|("CO" in location):
        return "Central"
    elif "WA" in location:
        return "Northwest"
    elif "TX" in location:
        return "Texas"
    elif ("NC" in location)|("GA" in location):
        return "Southeast"
    else:
        return "Other"
_


data['region'] = data['Location'].apply(locationToRegion)
        

In [ ]:
del data["Data Scientist"]

In [ ]:
dummy_jobFunction = pd.get_dummies(data['JobFunction'], prefix='JobFunction')
dummy_Seniority = pd.get_dummies(data['JobSeniority'], prefix='JobSeniority')
dummy_Region = pd.get_dummies(data['region'], prefix='Region')

In [ ]:
data

In [115]:
cols_to_keep = ['Company Name', 'Job Summary', 'Job Title',"Location","Salary Class","Class"]
merge = data[cols_to_keep].join(dummy_jobFunction.ix[:,])
merge1=merge.join(dummy_Seniority.ix[:, ])
merged=merge1.join(dummy_Region.ix[:, ])

merged

,Company Name,Job Summary,Job Title,Location,Salary Class,Class,JobFunction_Consultant,JobFunction_Cyber,JobFunction_Data scientist,JobFunction_Engineer,...,JobSeniority_Principle,JobSeniority_Senior,Region_California,Region_Central,Region_DMV,Region_Northeast,Region_Northwest,Region_Other,Region_Southeast,Region_Texas
0,Memorial Sloan Kettering Cancer Center,"Working together with members in the group, th...",Bioinformatics Data Scientist - Cancer Genomic...,"New York, NY 10065 (Upper East Side area)",Low,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,eTemp,High growth company focused on using data scie...,Mid - Senior Level Data Scientist,"New York, NY",Low,0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Comcast,Visible World is the leading provider of targe...,Data Scientist,"New York, NY",Low,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Samsung Accelerator,"Solid foundation in data structures, algorithm...",Machine Learning Engineer,"New York, NY",Low,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,DataKind,A mid-career data scientist - You have a succe...,Data Scientist,"New York, NY",Low,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,Bloomberg,Bloomberg's Global Data division houses an inc...,Machine Learning Engineer,"New York, NY",Low,0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,Crisis Text Line,Chief Data Scientist. The Data Scientist's rol...,Data Scientist,"New York, NY",Low,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,Paperless Post,Build production systems to collect and proces...,Data Scientist,"New York, NY",Low,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,DISH Network,A successful Data Scientist will have the foll...,Data Scientist,"New York, NY",Low,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,PlaceIQ,Data Scientists analyze PlaceIQ hyperlocal dat...,Data Scientist,"New York, NY",Low,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [119]:
#Base case is set to be that a Senior Consultant at California. The intercept of this case is 1, which
#represents a high income. Since it't the base case, we need to drop these variables.

del merged["JobSeniority_Senior"]
del merged["Region_California"]
del merged["JobFunction_Consultant"]

In [149]:
merged['intercept'] = 1

In [179]:

train_cols = merged.columns[6:]   
logit = sm.Logit(merged['Class'], merged[train_cols])
result = logit.fit_regularized()



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.502059715745
            Iterations: 177
            Function evaluations: 177
            Gradient evaluations: 177


In [163]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  Class   No. Observations:                 2906
Model:                          Logit   Df Residuals:                     2882
Method:                           MLE   Df Model:                           23
Date:                Thu, 30 Jun 2016   Pseudo R-squ.:                  0.2176
Time:                        16:20:49   Log-Likelihood:                -1459.0
converged:                       True   LL-Null:                       -1864.8
                                        LLR p-value:                1.196e-156
                                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
JobFunction_Cyber               30.3621   3.01e+06   1.01e-05      1.000     -5.91e+06  5.91e+06
JobFunction_Data scientist       0.3778      0.123      3.064      0.002     

In [158]:
print np.exp(result.params)

JobFunction_Cyber               1.534900e+13
JobFunction_Data scientist      1.459042e+00
JobFunction_Engineer            3.845034e+00
JobFunction_Machine learning    4.549566e+00
JobFunction_Marketing           8.448247e-01
JobFunction_Other               1.795170e+00
JobFunction_Research            4.857327e-01
JobFunction_Software            6.598454e+00
JobSeniority_Assistant          5.290086e-21
JobSeniority_Chief              1.724145e+01
JobSeniority_Director           4.542815e+00
JobSeniority_Entry              2.414952e-20
JobSeniority_Fellow             9.288688e-02
JobSeniority_Lead               1.687994e+00
JobSeniority_Manager            1.039177e+00
JobSeniority_Other              3.166252e-01
JobSeniority_Principle          1.276039e+13
Region_Central                  2.015123e-01
Region_DMV                      5.290781e-01
Region_Northeast                1.175839e+00
Region_Northwest                7.928010e-01
Region_Other                    6.245414e-07
Region_Sou

In [173]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                                  2.5%       97.5%            OR
JobFunction_Cyber             0.000000         inf  1.534900e+13
JobFunction_Data scientist    1.145804    1.857913  1.459042e+00
JobFunction_Engineer          2.844896    5.196775  3.845034e+00
JobFunction_Machine learning  2.537389    8.157420  4.549566e+00
JobFunction_Marketing         0.279116    2.557102  8.448247e-01
JobFunction_Other             1.389676    2.318982  1.795170e+00
JobFunction_Research          0.324319    0.727482  4.857327e-01
JobFunction_Software          2.617661   16.633019  6.598454e+00
JobSeniority_Assistant        0.000000         inf  5.290086e-21
JobSeniority_Chief            2.140967  138.847339  1.724145e+01
JobSeniority_Director         2.584057    7.986343  4.542815e+00
JobSeniority_Entry            0.000000         inf  2.414952e-20
JobSeniority_Fellow           0.011524    0.748671  9.288688e-02
JobSeniority_Lead             1.030139    2.765961  1.687994e+00
JobSeniority_Manager     

In [214]:
train_cols = merged.columns[6:]

logit = sm.Logit(merged['Class'], merged[train_cols])
result = logit.fit_regularized()

result.summary()

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.502059715745
            Iterations: 177
            Function evaluations: 177
            Gradient evaluations: 177


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Class   No. Observations:                 2906
Model:                          Logit   Df Residuals:                     2882
Method:                           MLE   Df Model:                           23
Date:                Fri, 01 Jul 2016   Pseudo R-squ.:                  0.2176
Time:                        11:26:05   Log-Likelihood:                -1459.0
converged:                       True   LL-Null:                       -1864.8
                                        LLR p-value:                1.196e-156
================================================================================================
                                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
JobFunction_Cyber               30.3621   3.01e+06   1.01e-05      1.000     -5.91e+06  5.91e+06
JobFunction_Data scientist       0.3778      0.123      3.064      0.002         0.136     0.619
JobFunction_Engineer             1.3468      0.154      8.762      0.000         1.046     1.648
JobFunction_Machine learning     1.5150      0.298      5.086      0.000         0.931     2.099
JobFunction_Marketing           -0.1686      0.565     -0.298      0.765        -1.276     0.939
JobFunction_Other                0.5851      0.131      4.479      0.000         0.329     0.841
JobFunction_Research            -0.7221      0.206     -3.504      0.000        -1.126    -0.318
JobFunction_Software             1.8868      0.472      4.000      0.000         0.962     2.811
JobSeniority_Assistant         -46.6885   5.41e+09  -8.63e-09      1.000     -1.06e+10  1.06e+10
JobSeniority_Chief               2.8473      1.064      2.675      0.007         0.761     4.933
JobSeniority_Director            1.5135      0.288      5.258      0.000         0.949     2.078
JobSeniority_Entry             -45.1700   2.81e+09  -1.61e-08      1.000     -5.51e+09  5.51e+09
JobSeniority_Fellow             -2.3764      1.065     -2.232      0.026        -4.463    -0.289
JobSeniority_Lead                0.5235      0.252      2.078      0.038         0.030     1.017
JobSeniority_Manager             0.0384      0.220      0.175      0.861        -0.392     0.469
JobSeniority_Other              -1.1500      0.109    -10.545      0.000        -1.364    -0.936
JobSeniority_Principle          30.1774   2.71e+06   1.11e-05      1.000     -5.31e+06  5.31e+06
Region_Central                  -1.6019      0.168     -9.540      0.000        -1.931    -1.273
Region_DMV                      -0.6366      0.174     -3.651      0.000        -0.978    -0.295
Region_Northeast                 0.1620      0.119      1.362      0.173        -0.071     0.395
Region_Northwest                -0.2322      0.154     -1.505      0.132        -0.534     0.070
Region_Other                   -14.2862   1861.713     -0.008      0.994     -3663.177  3634.605
Region_Southeast                -2.0410      0.245     -8.325      0.000        -2.521    -1.560
Region_Texas                    -2.1315      0.196    -10.864      0.000        -2.516    -1.747
================================================================================================
"""

In [227]:
from sklearn import linear_model, datasets
logreg = linear_model.LogisticRegression(C=1e5)
result2=logreg.fit(X, y)
result2




LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [228]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [238]:
X_test["actualHigh"] = y_test
X_test.head()
dfTrain = X_test

print dfTrain.columns

dfTrain['predictedHigh'] = result.predict( dfTrain[ dfTrain.columns[0:24] ] )

Index([u'JobFunction_Cyber', u'JobFunction_Data scientist',
       u'JobFunction_Engineer', u'JobFunction_Machine learning',
       u'JobFunction_Marketing', u'JobFunction_Other', u'JobFunction_Research',
       u'JobFunction_Software', u'JobSeniority_Assistant',
       u'JobSeniority_Chief', u'JobSeniority_Director', u'JobSeniority_Entry',
       u'JobSeniority_Fellow', u'JobSeniority_Lead', u'JobSeniority_Manager',
       u'JobSeniority_Other', u'JobSeniority_Principle', u'Region_Central',
       u'Region_DMV', u'Region_Northeast', u'Region_Northwest',
       u'Region_Other', u'Region_Southeast', u'Region_Texas', u'actualHigh'],
      dtype='object')


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [232]:
def scalePredictor(prob, threshold):
    if( prob >= threshold ):
        return 1
    return 0

In [240]:
i = 0.20
while i <= 0.50:
    
    dfTrain[ 'predictedHigh_{}'.format(i) ] = dfTrain.predictedHigh.apply(
        lambda prob: scalePredictor(prob,i) )
    
    print pd.crosstab(
        dfTrain['actualHigh'],
        dfTrain['predictedHigh_{}'.format(i)], 
        rownames=['High']
    )
    i += 0.01

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


predictedHigh_0.2    0    1
High                       
0                  302  285
1                   32  253
predictedHigh_0.21    0    1
High                        
0                   302  285
1                    33  252
predictedHigh_0.22    0    1
High                        
0                   303  284
1                    34  251
predictedHigh_0.23    0    1
High                        
0                   307  280
1                    36  249
predictedHigh_0.24    0    1
High                        
0                   317  270
1                    39  246
predictedHigh_0.25    0    1
High                        
0                   317  270
1                    40  245
predictedHigh_0.26    0    1
High                        
0                   317  270
1                    40  245
predictedHigh_0.27    0    1
High                        
0                   343  244
1                    46  239
predictedHigh_0.28    0    1
High                        
0                 

In [253]:
import numpy as np
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from sklearn import metrics

scores =cross_validation.cross_val_score(model, X, y, cv=6)
print "Cross-validated Scores:", scores
# Make cross validated predictions
predictions = cross_validation.cross_val_predict(model, X, y, cv=6)
#plt.scatter(y, predictions)
accuracy = metrics.accuracy_score(y, predictions)
print "Cross-Predicted Accuracy:", accuracy

Cross-validated Scores: [ 0.718107    0.59917355  0.47107438  0.61157025  0.72727273  0.76033058]
Cross-Predicted Accuracy: 0.647969717825
